---
title: "Source Code"
page-layout: full
---




# 1. Introduction & Objectives

This project moves beyond simple descriptive statistics to apply **Master's Level Machine Learning and Spatial Analysis** techniques to the College Scorecard dataset.

**Our Research Objectives:**

1\. **Earnings (Non-Linear):** Can we predict 10-year earnings using *Random Forest* to capture non-linear curriculum effects?

2\. **Graduation Rate (Interaction):** Does the impact of a "Tech Focus" depend on the *School Type* (Elite vs. Budget)?

3\. **Classification (SVM):** Can a *Support Vector Machine* accurately classify "High Value" schools?

4\. **Spatial Analysis (Geography):** Do "Education Deserts" (geographic isolation) negatively impact completion rates?

# 2. Data Preparation & Feature Engineering

We start by loading the data and applying advanced feature engineering: **PCA** (to reduce 30+ major variables) and **K-Means Clustering** (to identify school archetypes).




```{undefined}
# Load Libraries
required_packages <- c("tidyverse", "plotly", "broom", "scales", "randomForest", "glmnet", "caret", "rpart", "rpart.plot", "pdp", "e1071")
new_packages <- required_packages[!(required_packages %in% installed.packages()[, "Package"])]
if (length(new_packages)) install.packages(new_packages, repos = "http://cran.us.r-project.org")

library(tidyverse)
library(scales)
library(plotly)
library(randomForest)
library(caret)
library(rpart)
library(rpart.plot)
library(pdp)
library(e1071)

# Load Data
file_path <- "College_Scorecard_Raw_Data_10032025/Most-Recent-Cohorts-Institution.csv"

cols_to_keep <- c(
    "INSTNM", "STABBR", "CONTROL", "REGION", "LOCALE", "LATITUDE", "LONGITUDE",
    "ADM_RATE", "SAT_AVG", "UGDS", "COSTT4_A", "TUITIONFEE_IN", "TUITIONFEE_OUT",
    "MD_EARN_WNE_P10", "C150_4", "GRAD_DEBT_MDN", "PCTPELL", "INEXPFTE",
    "PCIP01", "PCIP03", "PCIP04", "PCIP05", "PCIP09", "PCIP10", "PCIP11",
    "PCIP12", "PCIP13", "PCIP14", "PCIP15", "PCIP16", "PCIP19", "PCIP22",
    "PCIP23", "PCIP24", "PCIP25", "PCIP26", "PCIP27", "PCIP29", "PCIP30",
    "PCIP31", "PCIP38", "PCIP39", "PCIP40", "PCIP41", "PCIP42", "PCIP43",
    "PCIP44", "PCIP45", "PCIP46", "PCIP47", "PCIP48", "PCIP49", "PCIP50",
    "PCIP51", "PCIP52", "PCIP54"
)

df <- read_csv(file_path, na = c("NULL", "PrivacySuppressed", "NA", "PS", ""), col_select = all_of(cols_to_keep), show_col_types = FALSE)

# Clean Data
numeric_cols <- setdiff(names(df), c("INSTNM", "STABBR", "CONTROL", "REGION", "LOCALE"))
df[numeric_cols] <- lapply(df[numeric_cols], as.numeric)
df$CONTROL <- as.factor(df$CONTROL)
df$REGION <- as.factor(df$REGION)
df_clean <- df %>%
    filter(!is.na(MD_EARN_WNE_P10), !is.na(C150_4), !is.na(COSTT4_A)) %>%
    na.omit()
```

```{undefined}

```




### 2.1 PCA & Clustering

Instead of using 38 separate variables for majors, we use **Principal Component Analysis (PCA)** to create "Curriculum Indices". We also use **K-Means** to group schools into 3 distinct "Archetypes".




```{undefined}
# PCA
pcip_cols <- grep("PCIP", names(df_clean), value = TRUE)
zero_var_cols <- pcip_cols[sapply(df_clean[, pcip_cols], var) == 0]
if (length(zero_var_cols) > 0) pcip_cols <- setdiff(pcip_cols, zero_var_cols)
pca_res <- prcomp(df_clean[, pcip_cols], scale. = TRUE)
df_clean <- cbind(df_clean, pca_res$x[, 1:5])

# Clustering
cluster_vars <- c("COSTT4_A", "UGDS", "ADM_RATE", "SAT_AVG")
df_scaled <- scale(df_clean[, cluster_vars])
set.seed(123)
kmeans_res <- kmeans(df_scaled, centers = 3)
df_clean$Cluster <- as.factor(kmeans_res$cluster)
```




------------------------------------------------------------------------

# 3. RQ1: Earnings Prediction (Non-Linear)

**Question:** Can we predict earnings based on curriculum and school stats?




```{undefined}
set.seed(123)
predictors <- c("PC1", "PC2", "PC3", "PC4", "PC5", "Cluster", "ADM_RATE", "SAT_AVG", "UGDS", "COSTT4_A", "CONTROL", "REGION")
rf_formula <- as.formula(paste("MD_EARN_WNE_P10 ~", paste(predictors, collapse = "+")))
rf_model <- randomForest(rf_formula, data = df_clean, ntree = 100, importance = TRUE)

# Partial Dependence Plot
pdp_pc1 <- pdp::partial(rf_model, pred.var = "PC1", train = df_clean)
p2 <- ggplot(pdp_pc1, aes(x = PC1, y = yhat)) +
    geom_line(color = "darkgreen", linewidth = 1.2) +
    labs(title = "Partial Dependence: Tech Focus (PC1) vs. Earnings", x = "PC1 (Tech Factor)", y = "Predicted Earnings") +
    theme_minimal()
ggplotly(p2)
```




------------------------------------------------------------------------

# 4. RQ2: Graduation Rate (Interaction Analysis)

**Question:** Does the benefit of a "Tech Curriculum" depend on the *Type of School*?




```{undefined}
# Interaction Model
lm_interaction <- lm(C150_4 ~ PC1 * Cluster + SAT_AVG + ADM_RATE + COSTT4_A + UGDS + CONTROL + PCTPELL, data = df_clean)
summary(lm_interaction)

# Interaction Plot
p3 <- ggplot(df_clean, aes(x = PC1, y = C150_4, color = Cluster)) +
    geom_point(alpha = 0.3) +
    geom_smooth(method = "lm", se = FALSE) +
    labs(title = "Interaction Effect: Curriculum x School Type", x = "PC1 (Tech Factor)", y = "Graduation Rate") +
    theme_minimal()
ggplotly(p3)
```




------------------------------------------------------------------------

# 5. RQ3: Classification (SVM vs. KNN)

**Question:** Can we classify "High Value" schools (High Earnings, Low Cost)?




```{undefined}
# Define High Value
earn_thresh <- quantile(df_clean$MD_EARN_WNE_P10, 0.66)
cost_thresh <- quantile(df_clean$COSTT4_A, 0.50)
df_clean <- df_clean %>% mutate(High_Value = as.factor(ifelse(MD_EARN_WNE_P10 > earn_thresh & COSTT4_A < cost_thresh, "Yes", "No")))

# Train SVM
svm_model <- svm(High_Value ~ SAT_AVG + ADM_RATE + UGDS + REGION + PC1 + Cluster, data = df_clean, kernel = "radial", cost = 10, scale = TRUE)

# Decision Boundary Visualization (PC1 vs PC2)
df_clean$Pred_SVM <- predict(svm_model, newdata = df_clean)
p4 <- ggplot(df_clean, aes(x = PC1, y = PC2, color = Pred_SVM)) +
    geom_point(alpha = 0.6) +
    labs(title = "SVM Decision Boundary (Tech vs Arts Factors)", x = "PC1 (Tech Factor)", y = "PC2 (Arts Factor)") +
    theme_minimal()
ggplotly(p4)
```




------------------------------------------------------------------------

# 6. RQ4: Spatial Analysis (Education Deserts)

**Question:** Does geographic isolation impact completion rates?




```{undefined}
deg2rad <- function(deg) {
    return(deg * pi / 180)
}
calculate_nearest_dist <- function(lat, lon, all_lats, all_lons) {
    R <- 6371 # Earth radius
    lat1 <- deg2rad(lat)
    lon1 <- deg2rad(lon)
    lats2 <- deg2rad(all_lats)
    lons2 <- deg2rad(all_lons)
    dlat <- lats2 - lat1
    dlon <- lons2 - lon1
    a <- sin(dlat / 2)^2 + cos(lat1) * cos(lats2) * sin(dlon / 2)^2
    c <- 2 * atan2(sqrt(a), sqrt(1 - a))
    d <- R * c
    d[d == 0] <- Inf
    return(min(d, na.rm = TRUE))
}

# Calculate Distances
distances <- numeric(nrow(df_clean))
lats <- df_clean$LATITUDE
lons <- df_clean$LONGITUDE
for (i in 1:nrow(df_clean)) distances[i] <- calculate_nearest_dist(lats[i], lons[i], lats, lons)
df_clean$Nearest_Dist <- distances

# Spatial Regression
lm_spatial <- lm(C150_4 ~ Nearest_Dist + INEXPFTE + CONTROL + REGION, data = df_clean)
summary(lm_spatial)

# Map Visualization
p5 <- ggplot(df_clean, aes(x = LONGITUDE, y = LATITUDE, color = C150_4, size = Nearest_Dist)) +
    geom_point(alpha = 0.7) +
    scale_color_viridis_c(option = "plasma", name = "Grad Rate") +
    borders("state", colour = "gray80", fill = NA) +
    labs(title = "Geography of Completion Rates", subtitle = "Size = Isolation (Distance)", x = "Longitude", y = "Latitude") +
    theme_minimal() +
    coord_fixed(1.3)
ggplotly(p5)
```




# 